In [2]:
import numpy as np
import skfuzzy as fuzz
from skfuzzy import control as ctrl
import streamlit as st

In [6]:
room_temperature = ctrl.Antecedent(np.arange(16, 36, 1), 'room_temperature')
temperature = ctrl.Antecedent(np.arange(16, 36, 1), 'temperature')
humidity = ctrl.Antecedent(np.arange(10, 90, 1), 'humidity')
diff_temperature = ctrl.Antecedent(np.arange(0, 5, 1), 'diff_temperature')

In [7]:
fan_speed = ctrl.Consequent(np.arange(0, 100, 1), 'fan_speed')
compressor_speed = ctrl.Consequent(np.arange(0, 100, 1), 'compressor_speed')

In [8]:

room_temperature['cold'] = fuzz.trimf(room_temperature.universe, [16, 21, 26])
room_temperature['warm'] = fuzz.trimf(room_temperature.universe, [20, 26, 32])
room_temperature['hot'] = fuzz.trimf(room_temperature.universe, [26, 30, 34])
# print("room_temperature", room_temperature.universe)

In [9]:
diff_temperature['high'] = fuzz.trimf(diff_temperature.universe, [2.5, 5, 5])
diff_temperature['medium'] = fuzz.trimf(diff_temperature.universe, [1.25, 2.5, 3.75])
diff_temperature['low'] = fuzz.trimf(diff_temperature.universe, [0, 0, 2.5])

In [10]:
humidity['low'] = fuzz.trimf(humidity.universe, [30, 45, 60])
humidity['medium'] = fuzz.trimf(humidity.universe, [45, 60, 75])
humidity['high'] = fuzz.trimf(humidity.universe, [60, 75, 90])
# print("humidity", humidity.universe)

In [16]:
fan_speed['low'] = fuzz.trimf(fan_speed.universe, [0, 0, 50])
fan_speed['medium'] = fuzz.trimf(fan_speed.universe, [25, 50, 75])
fan_speed['high'] = fuzz.trimf(fan_speed.universe, [50, 100, 100])
# print("fan_speed", fan_speed.universe)

In [17]:
compressor_speed['low'] = fuzz.trimf(compressor_speed.universe, [0, 0, 50])
compressor_speed['medium'] = fuzz.trimf(compressor_speed.universe, [25, 50, 75])
compressor_speed['high'] = fuzz.trimf(compressor_speed.universe, [50, 100, 100])


In [ ]:
# fan speed control rule
# rule 1a: if diff_temperature['high'] fan high speed, cooling high speed.
# rule 1b: if diff_temperature['medium'] fan medium speed, cooling medium speed.
# rule 1c: if diff_temperature['low'] fan medium low, cooling low speed.

# cooling speed control rule
# rule 1a: if 

In [18]:
def get_fan_speed_control_rules():
    rule1a = ctrl.Rule(diff_temperature['high'], fan_speed['high'])

    rule1b = ctrl.Rule(diff_temperature['medium'], fan_speed['medium'])

    rule1c = ctrl.Rule(diff_temperature['low'], fan_speed['low'])

    return [rule1a, rule1b, rule1c]


print("compressor_speed", compressor_speed.universe)

In [19]:
def get_compressor_speed_control_rules():
    rule2a = ctrl.Rule(diff_temperature['high'] | humidity['high'], compressor_speed['high'])
    rule2b = ctrl.Rule(diff_temperature['medium'] | compressor_speed['medium'], compressor_speed['medium'])
    rule2c = ctrl.Rule(diff_temperature['low'] | humidity['low'] | room_temperature['cold'], compressor_speed['low'])
    return [rule2a, rule2b, rule2c]


In [20]:
ac_ctrl = ctrl.ControlSystem(
    get_fan_speed_control_rules() + get_compressor_speed_control_rules()
)

In [21]:
def show_predict_page():
    st.title("Fuzzy Control On HVAC Final Project")
    st.subheader(f"Durham College Student: Wenping Wang ")
    st.write("""### please key in parameter to silumate the controls""")

    # Room Temperature Input
    in_rt = st.slider("Room Temperature (°C)", 16, 36, 16 )

    # temperature setpoint
    in_tt = st.slider("Temperature Setpoint(°C)", 16, 36, 22 )

    # Room Humidity Temperature
    in_hd = st.slider("Room Humidity (%)", 0, 70, 50 )

    # Difference between setpoint and room temperature
    in_diff = abs(in_rt - in_tt)

    speed = ctrl.ControlSystemSimulation(ac_ctrl)
    speed.input['room_temperature'] = int(in_rt)
    speed.input['diff_temperature'] = int(in_diff)
    speed.input['humidity'] = int(in_hd)
    speed.compute()
    fan_speed = speed.output['fan_speed']
    compressor_speed = speed.output['compressor_speed']
    ok = st.button("click to calculate the speed (%)")
    if ok:
      st.subheader(f"The fan speed is {fan_speed} % of full speed")
      st.subheader(f"The compressor speed is {compressor_speed} % of full speed")

In [ ]:

    
show_predict_page()
    
    